In [1]:
# We are importing all the stuff we need for the program to work
# If we crash here, we are missing some packages and need to install them



import numpy as np
from matplotlib import pyplot as plt
import pyaceqd.pulsegenerator as pg 
import os as os 
# this imports the PULSE modules we will use
from time import sleep
from Pulse_v2 import pulse_shaper_obj, attenuator, motor, spectrometer, power_meter, time_delay

# this imports the drivers for the hardware we will use
# replace with the actual drivers 
from Pulse_v2 import fake_motor # a closed loop linear motor, e.g. GMT 
from Pulse_v2 import fake_spectrometer # a spectrometer, e.g. Andor 
from Pulse_v2 import fake_power_meter # a power meter, e.g. thorlabs 
from Pulse_v2 import fake_attenuator # an attenuator, e.g. thorlabs 




We will now test the hardware drivers and if they contain the right commands. If not we need to modify the drivers, e.g. add the richt function. A future version will allow for user-specified commands. 

In [2]:
# first we will load the lab motor 
lab_motor = fake_motor()
# and then tranform it into a PULSE motor 
pulse_motor = motor(lab_motor, name='Fancy motor')
# this checks if lab_motor contains the functions "set_position(absolute_position)" and "close()"



Motor connected!
Motor name: Fancy motor
Motor device: <Pulse_v2.fake_motor object at 0x7f3284136f50>
Motor time of creation: 2024-10-01 12:04:38.725350


In [3]:
# We then check if it moves...
pulse_motor.set_position(20, excecute=True)
# the variable excecute determines if the motor should actually move and is default to False

Moving to position:  20


In [4]:
# ...and move back to 0 again.. if we want to...
pulse_motor.set_position(0, excecute=True)

Moving to position:  0


In [5]:
#... and if it closes again 
pulse_motor.close() 

Motor closed


Attenuator

In [6]:
# if that worked we will move on to the attenuator, again initializing the lab attenuator
lab_attenuator = fake_attenuator()
# and then tranform it into a PULSE attenuator
pulse_attenuator = attenuator(lab_attenuator, name='Fancy attenuator')
# this checks if lab_attenuator contains the functions "set_attenuation(attenuation)" and "close()"


Attenuator connected!
Attenuator name: Fancy attenuator
Attenuator device: <Pulse_v2.fake_attenuator object at 0x7f3284136da0>
Attenuator time of creation: 2024-10-01 12:04:54.891130


In [7]:
# then we will check if it attenuates
pulse_attenuator.set_attenuation(0.5, excecute=True)
sleep(2)
pulse_attenuator.set_attenuation(1, excecute=True)
sleep(2)
pulse_attenuator.set_attenuation(0, excecute=True)


# attenuators take values between 0 and 1, where 0 is no transmission and 1 is full transmission
# take care that your device also works within the same range

Setting attenuation to:  0.5
Setting attenuation to:  1
Setting attenuation to:  0


In [8]:
#... and if it closes again
pulse_attenuator.close()

Attenuator closed


Power Meter

In [9]:
# We now move on to the measurement devices, starting with the power meter
lab_power_meter = fake_power_meter()
pulse_power_meter = power_meter(lab_power_meter, name='fake')
# this checks if lab_power_meter contains the functions "get_power()" and "close()"



Power Meter connected!
Power Meter name: fake
Power Meter device: <Pulse_v2.fake_power_meter object at 0x7f32841364a0>
Power Meter time of creation: 2024-10-01 12:05:12.061692


In [10]:
# lets measure power and look at the output 
print(pulse_power_meter.get_power())
# we should see a list of length 2 with the first entry being the power measured in the lab and the second beiing reserved for the power of a simulation pulse (default to None)

got power: 1
[1, None]


In [11]:
# close the power meter
pulse_power_meter.close()

Power meter closed


In [12]:
# Finally we will look at the spectrometer
lab_spectrometer = fake_spectrometer(n_wl=1340, start_wl=774.15, end_wl=793.422)
pulse_spectrometer = spectrometer(lab_spectrometer, name='Fake spectrometer')
# this checks if lab_spectrometer contains the functions "get_spectrum()" and "close()" and already collects one spectrum to determine the wavelength range internally


Spectrometer connected!
Spectrometer name: Fake spectrometer
Spectrometer device: <Pulse_v2.fake_spectrometer object at 0x7f3284136ef0>
Spectrometer time of creation: 2024-10-01 12:05:22.549168


In [13]:
# again look at the output
output = pulse_spectrometer.get_spectrum()
print(output)
# we see a nested list where output[0][0] is the wavelength measured and output[0][1] is the recorded spectrum
# the second entry is reserved for the spectrum of a simulation pulse (default to None) following the same notation, e.g. output[1][0] and output[1][1]
# the third output the corresponds to emission calculated from a simulation. 
# These are part of the notebook: "DEMO_pulse_simulation.ipynb"

[[array([774.15      , 774.16439283, 774.17878566, ..., 793.39321434,
       793.40760717, 793.422     ]), array([67.11489013, 92.73216456, 43.81215759, ..., 65.42275871,
       22.21472787, 62.61396475])], [None, None], [None, None]]


In [14]:
# close the spectrometer
pulse_spectrometer.close()

Spectrometer closed


If all devices work as excpected we will move on and callibrate a pulse shaper. For this please setup your 4-f pulse shaper as you want to use it in the experiment. Make sure that your slit is sufficiently narrow for e.g. a TPE experiment. A change in slit width requires a new calibration run. 

Go to the skript "pulse_shaper_calibration.py" and return once you have a calibration file for the pulse shaper.

In a future version this will be steamlined.

In [15]:
# once we got the calibration we will create a pulse shaper object
# first we will need a PULSE motor 
lab_motor = fake_motor() # because we closed the motor before
pulse_motor = motor(lab_motor, name='Fancy motor')
# then we will transform it into a pulse shaper 
pulse_shaper_calibration = 'DEMO_calibration.txt'
pulse_shaper_pulse = pulse_shaper_obj(device=pulse_motor, calibration_file=pulse_shaper_calibration, name='Fancy pulse shaper')


Motor connected!
Motor name: Fancy motor
Motor device: <Pulse_v2.fake_motor object at 0x7f32883c68f0>
Motor time of creation: 2024-10-01 12:06:16.376157

Pulse Shaper connected!
Pulse Shaper name: Fancy pulse shaper
Pulse Shaper device: <Pulse_v2.motor object at 0x7f3284136d40>
Pulse Shaper time of creation: 2024-10-01 12:06:16.376393
Pulse Shaper calibration file: DEMO_calibration.txt


In [16]:
# lets setup an experiment consisting of a pulse shaper, an attenuator and a spectrometer 
lab_attenuator = fake_attenuator() # because we closed the attenuator before
pulse_attenuator = attenuator(lab_attenuator, name='Fancy attenuator')

lab_spectrometer = fake_spectrometer(n_wl=1340, start_wl=774.15, end_wl=793.422) # because we closed the spectrometer before
pulse_spectrometer = spectrometer(lab_spectrometer, name='Fake spectrometer')




Attenuator connected!
Attenuator name: Fancy attenuator
Attenuator device: <Pulse_v2.fake_attenuator object at 0x7f328831ee00>
Attenuator time of creation: 2024-10-01 12:06:27.713269

Spectrometer connected!
Spectrometer name: Fake spectrometer
Spectrometer device: <Pulse_v2.fake_spectrometer object at 0x7f3284136da0>
Spectrometer time of creation: 2024-10-01 12:06:27.714149


In [19]:
# each of the PULSE Devices hat a function called "open_control()" which opens a control object for the device
# Per default this opens a GUI to aid the experiments 
# As in the simulation notebook we must chain the controls to ensure the correct order of operations
attenuator_control = pulse_attenuator.open_control()
shaper_control = pulse_shaper_pulse.open_control(previous_control=attenuator_control)
spectrometer_control = pulse_spectrometer.open_control(previous_control=shaper_control)
# and we need to tell one of the controllers to start the mainloop for the gui
shaper_control.start_gui()

Current attenuation:  0.5
Step size:  0.02
Large step size:  10
Initial attenuation:  None
Open GUI:  True
Parent window:  None
Previous control:  None
Pulseshaper: Fancy pulse shaper
Current position: 794.2148352443684
Current step size: 0.1
Spectrometer Control
Spectrometer Object: Fake spectrometer
Open GUI: True
Display Experiment: True
Display Pulse: False
Display Simulation: False
Autoscale: True
hier könnte ihre GUI stehen!
Moving to position:  1.658073574210988
Setting attenuation to:  0.48
Setting attenuation to:  0.45999999999999996
Setting attenuation to:  0.48
Setting attenuation to:  0.5
Moving to position:  2.1022727272727626
Moving to position:  2.546471880334537
Moving to position:  2.990671033396312
Moving to position:  3.434870186458086
Moving to position:  2.990671033396312
Moving to position:  2.546471880334537
Moving to position:  2.1022727272727626
Motor closed
Fancy pulse shaper closed
Attenuator closed
Window closed


 I hope everything worked so far. Although cool this is not particularly useful.. yet. Thats why we will now use these tools to optimize a TPE experiment
 

In [20]:
# Since closing the GUIs disconnects the devices, we will have to reconnect them
lab_motor = fake_motor() # because we closed the motor before
pulse_motor = motor(lab_motor, name='Fancy motor')
# then we will transform it into a pulse shaper 
pulse_shaper_calibration = 'DEMO_calibration.txt'
pulse_shaper_pulse = pulse_shaper_obj(device=pulse_motor, calibration_file=pulse_shaper_calibration, name='Fancy pulse shaper')

lab_attenuator = fake_attenuator() # because we closed the attenuator before
pulse_attenuator = attenuator(lab_attenuator, name='Fancy attenuator')

lab_spectrometer = fake_spectrometer(n_wl=1340, start_wl=774.15, end_wl=793.422) # because we closed the spectrometer before
pulse_spectrometer = spectrometer(lab_spectrometer, name='Fake spectrometer') 

# open the respective control objects 

attenuator_control = pulse_attenuator.open_control()
shaper_control = pulse_shaper_pulse.open_control(previous_control=attenuator_control)
spectrometer_control = pulse_spectrometer.open_control(previous_control=shaper_control) 


#Before launching the GUIs we will load a package called control optimizer
import control_optimizer as co

optimizer = co.control_optimizer(device_control=[attenuator_control,shaper_control], measururement_control=spectrometer_control, measurement_kind='spectrometer')
# the input "device_control" can take a list of all PULSE controllers corresponding to a Pulse shaping device (attenuator, shaper, time delay stage, in the future also waveplates ect.) 
# the input "measurement_control" can take measurement devices as input. Currently only the spectrometer is supported, but SNSPDs ect. will follow  
# don't forget to start the mainloop for the optimizer

optimizer.start_gui()


Motor connected!
Motor name: Fancy motor
Motor device: <Pulse_v2.fake_motor object at 0x7f323234f760>
Motor time of creation: 2024-10-01 12:09:24.175213

Pulse Shaper connected!
Pulse Shaper name: Fancy pulse shaper
Pulse Shaper device: <Pulse_v2.motor object at 0x7f323234ebf0>
Pulse Shaper time of creation: 2024-10-01 12:09:24.175417
Pulse Shaper calibration file: DEMO_calibration.txt

Attenuator connected!
Attenuator name: Fancy attenuator
Attenuator device: <Pulse_v2.fake_attenuator object at 0x7f323234ee30>
Attenuator time of creation: 2024-10-01 12:09:24.176805

Spectrometer connected!
Spectrometer name: Fake spectrometer
Spectrometer device: <Pulse_v2.fake_spectrometer object at 0x7f323234f2b0>
Spectrometer time of creation: 2024-10-01 12:09:24.176910
Setting attenuation to:  0.5
Current attenuation:  0.5
Step size:  0.02
Large step size:  10
Initial attenuation:  None
Open GUI:  True
Parent window:  None
Previous control:  None
Moving to position:  2.1022727272727626
Pulseshape